In [2]:
import os
from PIL import Image
import numpy as np


In [3]:

class NeuralNetwork:
    def __init__(self, layer_sizes, learning_rate=0.01):
        self.num_layers = len(layer_sizes)
        self.learning_rate = learning_rate
        self.weights = []
        self.biases = []

        # Initialize weights and biases
        for i in range(self.num_layers - 1):
            self.weights.append(np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * 0.01)
            self.biases.append(np.zeros((1, layer_sizes[i + 1])))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        return x * (1 - x)

    def compute_loss(self, y_true, y_pred):
        return np.mean((y_true - y_pred) ** 2)

    def forward(self, X):
        self.activations = [X]
        for i in range(self.num_layers - 1):
            Z = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]
            A = self.sigmoid(Z)
            self.activations.append(A)
        return self.activations[-1]

    def backward(self, y):
        m = y.shape[0]
        deltas = [self.activations[-1] - y]
        for i in range(self.num_layers - 2, 0, -1):
            delta = np.dot(deltas[-1], self.weights[i].T) * self.sigmoid_derivative(self.activations[i])
            deltas.append(delta)
        deltas.reverse()

        for i in range(self.num_layers - 1):
            dW = np.dot(self.activations[i].T, deltas[i]) / m
            db = np.sum(deltas[i], axis=0, keepdims=True) / m
            self.weights[i] -= self.learning_rate * dW
            self.biases[i] -= self.learning_rate * db

    def train(self, X, y, epochs, X_val=None, y_val=None):
        for epoch in range(epochs):
            output = self.forward(X)
            self.backward(y)
            if epoch % 100 == 0:
                loss = self.compute_loss(y, output)
                print(f"Epoch {epoch}, Loss: {loss}")
                if X_val is not None and y_val is not None:
                    val_output = self.forward(X_val)
                    val_loss = self.compute_loss(y_val, val_output)
                    print(f"Validation Loss: {val_loss}")

    def evaluate(self, X, y):
        predictions = self.forward(X)
        correct = np.sum(np.argmax(predictions, axis=1) == np.argmax(y, axis=1))
        accuracy = correct / y.shape[0]
        return accuracy





In [4]:
def load_data(data_dir, categories, image_size=(64, 64)):
    X_data = []
    labels = []

    for folder in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder)

        if not os.path.isdir(folder_path):
            continue

        if folder not in categories:
            print(f"Skipping unknown category folder: {folder}")
            continue

        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)

            try:
                image = Image.open(file_path).convert('L')
                image = image.resize(image_size)
                image_array = np.array(image) / 255.0
                X_data.append(image_array.flatten())

                label = np.zeros(len(categories))
                label[categories.index(folder)] = 1
                labels.append(label)
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")

    return np.array(X_data), np.array(labels)



In [ ]:

# Paths to directories
train_dir = "Task 4.0/Train"
test_dir = "Task 4.0/Test"

# Categories
categories = ["Jade", "James", "Jane", "Joel", "Jovi"]

# Load training and test data
X_train, y_train = load_data(train_dir, categories)
X_test, y_test = load_data(test_dir, categories)

# Debugging output for dataset dimensions
print(f"Training data shape: {X_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Test labels shape: {y_test.shape}")

# Neural network configuration
input_size = X_train.shape[1]
hidden_size = 64
output_size = len(categories)
learning_rate = 0.1
epochs = 1000

# Initialize and train the neural network
nn = NeuralNetwork(layer_sizes=[input_size,hidden_size, output_size], learning_rate=learning_rate)
nn.train(X_train, y_train, epochs)

# Evaluate on test data
accuracy = nn.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Training data shape: (4000, 4096)
Training labels shape: (4000, 5)
Test data shape: (1000, 4096)
Test labels shape: (1000, 5)
Epoch 0, Loss: 0.2539336526393231
Epoch 100, Loss: 0.1245173131657908
Epoch 200, Loss: 0.08581676736743264
Epoch 300, Loss: 0.07021880254831078
Epoch 400, Loss: 0.06050313309935417
Epoch 500, Loss: 0.050223151819127836
Epoch 600, Loss: 0.04124496376299186
Epoch 700, Loss: 0.03486742923935077
Epoch 800, Loss: 0.030518461062498642
Epoch 900, Loss: 0.027378374153730317
Test Accuracy: 92.90%
